In [1]:
import TableParser
import pandas as pd
import tempfile, zipfile
import os
import numpy as np

In [129]:
class TestTableParser(TableParser.TableParserItem):
    def __init__(self):
        pass
    
    def parse(self, series: pd.Series):
        mistake = {
            'Jan':'01','Feb': '02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sept':'09','Oct':'10','Nov':'11','Dec':'12',
            'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5',
            r'[ -/]|[:-@]|[\[-_]|[{-~]': '-',
        }
        for key, val in mistake.items():
            series = series.str.strip().str.replace(key, val, regex=True)
        return (pd.to_datetime(series, errors="ignore"), [])

    def fix_xlsx(self, in_file):
        tmpfd, tmp = tempfile.mkstemp(dir=os.path.dirname(in_file))
        os.close(tmpfd)
        filename = '[Content_Types].xml'
        data = ''
        with zipfile.ZipFile(in_file, 'r') as zin:
            with zipfile.ZipFile(tmp, 'w') as zout:
                for item in zin.infolist():
                    if item.filename != filename:
                        zout.writestr(item, zin.read(item.filename))
                    else:
                        data = zin.read(filename).decode()
        os.remove(in_file)
        os.rename(tmp, in_file)
        data = data.replace('/xl/sharedStrings.xml', '/xl/SharedStrings.xml')
        with zipfile.ZipFile(in_file, mode='a', compression=zipfile.ZIP_DEFLATED) as zf:
            zf.writestr(filename, data)

    ###############################################
    # to find the x,y coordinate of row with date #
    
    def find_coordinate(self, x, y,df): # and defind axis
        test_list =[[0,0]]
        count_X_axis,count_Y_axis,main_axis = 0,0,"Y axis"
        
        for i in range(1,df.shape[0]):
            if len(test_list) == 11: 
                break
            for j in range(df.shape[1]):
                if  type(str(df.iloc[i,j])) == type("string") and len(str(df.iloc[i,j])) > 5:
                    if type(testTableParser.parse(pd.Series([str(df.iloc[i,j])]))[0][0]) == type(pd.to_datetime('31.01.2002')):
                        test_list.append([i,j])
                        if test_list[-1][1] > test_list[-2][1]:
                            count_X_axis += 1
                        if test_list[-1][0] > test_list[-2][0]:
                            count_Y_axis += 1  
                        if len(test_list) == 11:
                            break
        if count_X_axis > count_Y_axis:
            main_axis = "X axis"
#         print(main_axis,test_list)
        return test_list[1][0],test_list[1][1],main_axis
    
   
    
    def y_axis_solution(self,df,data_row_num,data_col_num):
        table = df.iloc[data_row_num:, data_col_num:].reset_index(drop=True)  # <--- all useful data
        date = df.iloc[data_row_num:,data_col_num].astype(str)
        date = self.parse(date)
        table.iloc[0:,0] = date[0]
#             data_row_num,data_col_num = 35,4
        min_height = 0
        if data_row_num > 3:min_height = data_row_num- 3
        elif data_row_num< 3 and data_row_num >1: min_height = data_row_num -1
        list_index =[]
        for i in range(data_col_num,df.shape[1]):
            buffer = ""
            if pd.isnull(df.iloc[min_height,i]) and len(list_index)!= 0:
                buffer = "upWord_"
            for j in range(min_height,data_row_num):
                buffer += str(df.iloc[j,i]).replace('\n',' ')+ "__"
            list_index.append(buffer.replace("nan__","")[:-2])
            if list_index[-1][:7] == "upWord_":
                index = list_index[-2].find("_")
                list_index[-1] = list_index[-2][:index+2]+list_index[-1][7:]
        list_index = pd.Series(list_index)
        table.columns = list_index   
        return table
    
    def x_axis_solution(self,df,data_row_num,data_col_num):
#         print(data_row_num,data_col_num)
        table = df.iloc[data_row_num:, data_col_num:].reset_index(drop=True)  # <--- all useful data
        date = df.iloc[data_row_num, data_col_num:].astype(str)  # <--- Series with date (01.03.2016;...)
        date = testTableParser.parse(date)
        table.iloc[0] = date[0]
        #             data_row_num,data_col_num = 4,3


        min_deep = 0
        if data_col_num > 3:min_deep = data_col_num - 3
        elif data_col_num< 3 and data_col_num >1: min_deep = data_col_num -1
        list_index =[]
        for i in range(data_row_num,df.shape[0]):
            buffer = ""
            if pd.isnull(df.iloc[i,min_deep]) and len(list_index)!= 0:
                buffer = "upWord_"
            for j in range(min_deep,data_col_num):
                buffer +=str(df.iloc[i,j]).replace('\n',' ')+"__"
            list_index.append(buffer.replace("nan__","").strip())
            if list_index[-1][:7] == "upWord_":
                index = list_index[-2].find("_")
                list_index[-1] = list_index[-2][:index+2]+list_index[-1][7:]
        list_index = pd.Series(list_index).str[:-2]
        return table.set_index(list_index)
    
    def all_action(self,df):
        data_row_num, data_col_num,main_axis = self.find_coordinate(0,0,df) # <-- find coordinate of dates
#         print(data_row_num, data_col_num,main_axis)
        if main_axis == "X axis":
            return self.x_axis_solution(df,data_row_num,data_col_num)
        elif main_axis == "Y axis":
            return self.y_axis_solution(df,data_row_num,data_col_num)
        return df

In [135]:
testTableParser = TestTableParser()
# testTableParser.fix_xlsx('file1.xlsx')
df = pd.read_excel("file22.xlsx")
testTableParser.all_action(df.T)

,3,4,5,6,7,8,9,10,11,12,...,521,522,523,524,525,526,527,528,529,530
Показатель__№ корпуса__Дата,2019-12-28-00-00-00,2019-12-29-00-00-00,2019-12-30-00-00-00,2019-12-31-00-00-00,2020-01-01-00-00-00,2020-01-02-00-00-00,2020-01-03-00-00-00,2020-01-04-00-00-00,2020-01-05-00-00-00,2020-01-06-00-00-00,...,2021-05-05-00-00-00,2021-05-06-00-00-00,2021-05-07-00-00-00,2021-05-08-00-00-00,2021-05-09-00-00-00,2021-05-10-00-00-00,2021-05-11-00-00-00,2021-05-12-00-00-00,2021-05-13-00-00-00,2021-05-14-00-00-00
"1 корпус (кур)__Поголовье, гол",8471,8467,8461,8451,8446,8444,8439,8438,8438,8438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1 корпус (кур),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1 корпус (кур)__Выбраковка, гол",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1 корпус (кур),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15 корпус__Продуктивность,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15 корпус__Вес яйца,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15 корпус__Вода,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"15 корпус__t, С утро",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# pd.to_datetime('01.02.2002')
df.shape

(182, 153)

In [5]:
df.shape


(182, 153)

In [6]:
type(df.iloc[2,4])

int

In [ ]:
print("as_asd".find("_"))

In [ ]:
df = pd.read_excel("file22.xlsx")
xls = pd.ExcelFile('file22.xlsx')
xls.sheet_names

In [ ]:
sheet_to_df_map = {}
for sheet_name in xls.sheet_names:
    sheet_to_df_map[sheet_name] = xls.parse(sheet_name)

In [ ]:
list(sheet_to_df_map.values())[0]

In [ ]:
type(df.iloc[3,2])

In [ ]:
type(pd.to_datetime('01.01.2021'))